### Imports

In [63]:
from pylab import *
%matplotlib inline
rcParams['figure.figsize'] = (15.0, 30.0)

* Prepare paths

In [64]:
import sys
sys.path.append('..')


### General data structures

* Preload data

In [65]:
from dataset import get_positive_patches, get_negative_patches, get_labels, load_image
from common import dataset, utils
import numpy as np
import pandas as pd

labels_data, data = dataset.labels_data, dataset.data


## This one contains separatedly cars, vans...
#vehicle_types = set(dataset.map_category.values())
#vehicle_types.add('Empty')

map_category = {'A':'Vehicle', 'B':'Vehicle', 'C':'Vehicle', 'D':'Vehicle', 'E':'Vehicle', 'F':'Vehicle', 'G':'Vehicle', 'H':'Vehicle', 'I':'Vehicle'}
labels_data = pd.read_csv('../data/trainingObservations.csv')
data = get_labels(labels_data, map_category)
vehicle_types = set(map_category.values())

* Split train and test

In [66]:
#test_images = 60
#cases = list(data.keys())
#np.random.shuffle(cases)
#train_cases, test_cases = cases[test_images:], cases[:test_images]
#np.save('../data/patch_images/train_cases', train_cases.npy)
#np.save('../data/patch_images/test_cases', test_cases.npy)
train_cases = np.load('../data/patch_images/train_cases.npy')
test_cases  = np.load('../data/patch_images/test_cases.npy')

### Prepare the dataset and the generator

Save positive and negative images in different folders

In [68]:
import scipy.misc
import os

for traintest_case in (['test']):#(['train','test']):
    number_of_negative_per_image = 50

    ## Delete all the folders
    for key in list(vehicle_types):
        os.system('rm -rf ../data/patch_images/%s/%s || True' % (traintest_case, key))

    # Create new folders
    for key in list(vehicle_types):
        os.system('mkdir ../data/patch_images/%s/%s || True' % (traintest_case, key))

    # Generate positive samples and save them in the proper folder
    if traintest_case == 'train':
        files = train_cases
    elif traintest_case == 'test':
        files = test_cases
    else:
        raise 'Inutil! Train o test!'
        
    for case in files:
        for key in list(vehicle_types):
            pos = get_positive_patches(dataset.load_image(case), data[case], 110, [key])    
            for j in range(pos.shape[0]):
                scipy.misc.imsave('../data/patch_images/%s/%s/%s_%d.jpg' % (traintest_case,key,case,j), pos[j])

    # Generate negative samples and save them in the proper folder
    os.system('rm -rf ../data/patch_images/%s/Empty' % traintest_case)
    os.system('mkdir ../data/patch_images/%s/Empty' % traintest_case)
    for case in train_cases:
        pos = get_negative_patches(dataset.load_image(case), data[case], 110, number_of_negative_per_image)
        for j in range(pos.shape[0]):
            scipy.misc.imsave('../data/patch_images/%s/%s/%s_%d.jpg' % (traintest_case,'Empty',case,j), pos[j])

vehicle_types.add('Empty')

Test the generator

In [7]:
vehicle_types.add('Empty')

In [69]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_image_dim_ordering('th')

def generate_training_cases(batch_size, case):
    if case not in ('train','test'):
        raise "Only train o test"
    datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, horizontal_flip = True, zoom_range = 0.02, rotation_range=180)
    a = 0
    for X_batch, y_batch in datagen.flow_from_directory('../data/patch_images/%s'%s, target_size = [110,110], class_mode = 'binary', classes = vehicle_types, batch_size=batch_size):
        yield X_batch, y_batch
        
#a = 0
#for X_batch, y_batch in generate_training_cases(32):
#    #utils.multiplot(X_batch.transpose([0,2,3,1]), [10,5])
#    print(y_batch.sum(axis = 0))
#    a+=1
#    if a == 10:
#        break

train_generator = generate_training_cases(32, 'train')
test_generator = generate_training_cases(32, 'test')
print('generator_ready')

generator_ready


### Train a resnet50

In [9]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard


OUTPUT_MODEL = '../data/models/vehicle_empty_discriminator.hdf5'  # OUTPUT_MODEL = wp + 'personal/jm_patches_train_v06_local.hdf5'
LOGS_PATH    = '../logs/%s' % str('vehicle_empty_resnet_v0')

#LOGS_PATH = wp + 'logs/%s' % str(int(time()))
if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)
    
K.set_image_dim_ordering('th')
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M:%S')

tb = TensorBoard(log_dir=LOGS_PATH, histogram_freq=1, write_graph=False, write_images=False)  # replace keras.callbacks.TensorBoard
model_checkpoint = ModelCheckpoint(OUTPUT_MODEL, monitor='loss', save_best_only=True)



# Load model
model = ResnetBuilder().build_resnet_18((3,110,110),1)
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy','fmeasure'])
# model.load_weights(OUTPUT_MODEL)


batch_size = 32
model.fit_generator(generator=train_generator,
                    samples_per_epoch=500,  # make it small to update TB and CHECKPOINT frequently
                    nb_epoch=500,
                    verbose=1,
                    #class_weight={0:1., 1:4.},
                    callbacks=[tb, model_checkpoint],
                    validation_data=test_generator,  # TODO: is_training=False
                    nb_val_samples=64,
                    max_q_size=256,
                    nb_worker=1)  # a locker is needed if increased the number of parallel workers


INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.


INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.


INFO:tensorflow:Summary name batchnormalization_1_gamma:0 is illegal; using batchnormalization_1_gamma_0 instead.


...

INFO:tensorflow:Summary name dense_1_W:0 is illegal; using dense_1_W_0 instead.


INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.


INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.


Epoch 1/500
Found 33139 images belonging to 2 classes.
 128/1280 [==>...........................] - ETA: 528s - loss: 1.4907 - acc: 0.8359 - fmeasure: 0.4058

KeyboardInterrupt: 

### Test on an image

In [45]:
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d

patch_size = 110
pred_per_row = []
for i_row in range(2000-patch_size):
    patches = extract_patches_2d(load_image(train_cases[0])[i_row:i_row+110,:], [patch_size,patch_size]).transpose([0,3,1,2])
    out = model.predict(patches, batch_size = 32).flatten()
    pred_per_row.append(out)

#out = out.reshape([out.shape[0],1,1])

In [24]:
#reconstructed = reconstruct_from_patches_2d(out, [120-patch_size+1,120-patch_size+1])
#imshow(reconstructed, cmap = cm.Greys)